# Healthy Ride Data Exploration

<img src='https://healthyridepgh.com/wp-content/uploads/sites/3/2016/09/Healthy-Ride-Logo.Stacked-01.png' width='50%'>

<big><big><big>PGH Data Science Meetup</big></big></big>

<b>Show and Tell</b>

*December 6, 2017*

* Robert Lucente, *Pipeline*
* Chris Sternberger, *BAE Systems*
* Melinda Angeles, *Allegheny County*
* <u>Albert DeFusco</u>, *Anaconda, Inc.*

# Table of Contents
* [Healthy Ride Data Exploration](#Healthy-Ride-Data-Exploration)
	* [Get Data](#Get-Data)
	* [2016 Rides](#2016-Rides)
	* [Stations](#Stations)
	* [Cleaning](#Cleaning)
		* [Geocoding](#Geocoding)
	* [Again for 2017](#Again-for-2017)
	* [Ride difficulty](#Ride-difficulty)
		* [elevation](#elevation)
		* [distance](#distance)
		* [grade](#grade)
		* [outliers](#outliers)
	* [Google Distance Matrix](#Google-Distance-Matrix)
* [Datetime](#Datetime)


## Get Data

We'll do a little web scraping here to get the Zip files.

In [1]:
import os
import shutil

if os.path.exists('./data'):
    shutil.rmtree('./data/')
    
os.mkdir('./data')

<big><big>[Requests: HTTP for Humans](http://docs.python-requests.org/en/master/)</big></big> | <big><big>[BeautifulSoup: HTML parsing](https://www.crummy.com/software/BeautifulSoup/)</big></big>
:-------------------------:|:-------------------------:
<img src='http://docs.python-requests.org/en/master/_static/requests-sidebar.png' width='20%'>  |  <img src='https://singbookswithemily.files.wordpress.com/2015/03/beautiful-soup-can-w-mock-turtle-coin.jpg' width='40%'>

In [2]:
import requests
from bs4 import BeautifulSoup
from os.path import basename, join

response = requests.get('http://healthyridepgh.com/data')
soup = BeautifulSoup(response.content, 'lxml')

buttons = soup.find_all('a', attrs={'class':'btn btn-primary'})
links = [b.get('href') for b in buttons]

for link in links:
    response = requests.get(link)
    if response.ok:
        fname = basename(link)
        print('Downloading {}'.format(fname))
        with open(join('data',fname), 'wb') as f:
            f.write(response.content)

os.listdir('./data')

True

Finally, we'll extract the archives.

In [5]:
from zipfile import ZipFile

for file in os.listdir('./data'):
    with ZipFile(join('./data',file)) as z:
        z.extractall('./data')

In [38]:
for root, dirs, files in os.walk('./data'):
    if not dirs:
        print(root+'/')
        for f in files:
            print(' '*14,f)

./data/2017-Q1/
               Healthy Ride Rentals 2017-Q1.csv
               HealthyRideStations2017.csv
               desktop.ini
./data/2016-Q1/
               HealthyRide Rentals 2016 Q1.csv
               HealthyRideStations2016.csv
               desktop.ini
./data/2015-Q2/
               HealthyRideStations2015.csv
               HealthyRide Rentals 2015 Q2.csv
               desktop.ini
./data/2015-Q3/
               HealthyRideStations2015.csv
               HealthyRide Rentals 2015 Q3.csv
               desktop.ini
./data/2015-Q4/
               HealthyRideStations2015.csv
               HealthyRide Rentals 2015 Q4.csv
               desktop.ini
./data/2017-Q2/
               Healthy Ride Rentals 2017-Q2.csv
               HealthyRideStations2017.csv
./data/2016-Q2/
               HealthyRide Rentals 2016 Q2.csv
               HealthyRideStations2016.csv
               desktop.ini
./data/2016-Q3/
               HealthyRide Rentals 2016 Q3.csv
               HealthyRideStati

## 2016 Rides


<img src='https://pandas.pydata.org/_static/pandas_logo.png' width='80%'>

http://pandas.pydata.org

"Python and data analysis" and "panel data"

Read all of the 2016 data into a single Pandas DataFrame object it's only 70,000 rows.

In [120]:
import pandas as pd
from glob import glob
pd.options.display.max_rows = 8

files = glob('./data/2016-Q*/HealthyRide Rentals 2016 Q*.csv')
rides2016 = pd.concat([pd.read_csv(f) for f in files], ignore_index=True)

for c in 'Starttime','Stoptime':
    rides2016[c] = pd.to_datetime(rides2016[c], format='%m/%d/%Y %H:%M')
    
rides2016.head()

,Trip id,Starttime,Stoptime,Bikeid,Tripduration,From station id,From station name,To station id,To station name,Usertype
0,15335599,2016-01-01 01:44:00,2016-01-01 02:01:00,70294,1068,1026,Penn Ave & S Whitfield St,1032,Walnut St & College St,NaN
1,15335629,2016-01-01 02:39:00,2016-01-01 02:53:00,70360,892,1029,Alder St & S Highland Ave,1021,Taylor St & Liberty Ave,NaN
2,15336195,2016-01-01 10:02:00,2016-01-01 10:06:00,70369,245,1029,Alder St & S Highland Ave,1028,Penn Ave & Putnam St (Bakery Square),NaN
3,15336282,2016-01-01 10:39:00,2016-01-01 10:50:00,70304,683,1041,Fifth Ave & S Bouquet St,1047,S 22nd St & E Carson St,NaN
4,15336307,2016-01-01 10:51:00,2016-01-01 11:14:00,70345,1346,1047,S 22nd St & E Carson St,1002,Third Ave & Wood St,NaN


## Stations

Positions of each station are stored separately. 2017 probably has the most up to date stations.

In [122]:
stations = pd.read_csv('data/2017-Q1/HealthyRideStations2017.csv',encoding='iso-8859-1', index_col=0).sort_index()
stations

,Station Name,# of Racks,Latitude,Longitude
Station #,,,,
1000,Liberty Ave & Stanwix St,16,40.441326,-80.004679
1001,Forbes Ave & Market Square,19,40.440877,-80.003080
1002,Third Ave & Wood St,15,40.439030,-80.001860
1003,First Ave & Smithfield St (Art Institute),15,40.437200,-80.000375
...,...,...,...,...
1048,S 18th St & Sidney St,16,40.429338,-79.980684
1049,S 12th St & E Carson St,19,40.428661,-79.986358
1050,Healthy Ride Hub,2,40.461817,-79.968002
1061,33rd St and Penn Ave,2,40.462026,-79.968114


## Harmonizing

The rides file has *MORE* station IDs than are listed in the stations file.

In [123]:
from_ids = rides2016['From station id'].drop_duplicates()
to_ids = rides2016['To station id'].drop_duplicates()

station_ids = pd.concat([from_ids, to_ids]).drop_duplicates()
len(station_ids)

53

What is station 1060???

In [124]:
missing_stations = station_ids[~station_ids.isin(stations.index)].tolist()
missing_stations

[1060]

Let's go get its name.

In [125]:
missing_names = rides2016.loc[rides2016['From station id'].isin(missing_stations), 'From station name'].drop_duplicates()
missing_names

46569    Open Streets West End
Name: From station name, dtype: object

### Geocoding 

<img src='img/gmaps.png'>


Open Streets West End looks like a [temporary station](http://openstreetspgh.org/schedule-july-2016/#WABASH).

*Let's got find it!*

[GoogleMaps](https://github.com/googlemaps/google-maps-services-python) makes interacting with the API easy once you have a key.

**Note**: you will not be able to install `my_secrets`. These are my private API keys.

In [126]:
import googlemaps
from my_secrets.keys import google_gmap

c_map = googlemaps.Client(key=google_gmap)
wabash_and_main = c_map.geocode('Wabash and Main St. West End Pittsburgh, PA')[0]['geometry']['location']

In [127]:
import numpy as np

new_station = {
    'Station Name':'Open Streets West End',
    '# of Racks':np.nan,
    'Latitude':wabash_and_main['lat'],
    'Longitude':wabash_and_main['lng']
}

stations.loc[1060] = new_station

In [128]:
stations

,Station Name,# of Racks,Latitude,Longitude
Station #,,,,
1000,Liberty Ave & Stanwix St,16.0,40.441326,-80.004679
1001,Forbes Ave & Market Square,19.0,40.440877,-80.003080
1002,Third Ave & Wood St,15.0,40.439030,-80.001860
1003,First Ave & Smithfield St (Art Institute),15.0,40.437200,-80.000375
...,...,...,...,...
1049,S 12th St & E Carson St,19.0,40.428661,-79.986358
1050,Healthy Ride Hub,2.0,40.461817,-79.968002
1061,33rd St and Penn Ave,2.0,40.462026,-79.968114
1060,Open Streets West End,NaN,40.440910,-80.034810


## Again for 2017

In [129]:
files = glob('./data/2017-Q*/Healthy Ride Rentals 2017-Q*.csv')
rides2017 = pd.concat([pd.read_csv(f, encoding='iso-8859-1') for f in files], ignore_index=True)


for c in 'Starttime','Stoptime':
    rides2017[c] = pd.to_datetime(rides2017[c], format='%m/%d/%Y %H:%M')

from_ids = rides2017['From station id'].drop_duplicates()
to_ids = rides2017['To station id'].drop_duplicates()

station_ids2017 = pd.concat([from_ids, to_ids]).drop_duplicates()

<big><big><big><i>Whoa!!!</i></big></big></big>

There are 32 missing stations.

In [130]:
missing_stations = station_ids2017[~station_ids2017.isin(stations.index)]
len(missing_stations)

32

Some of these don't look like Pittsburgh stations.

Let's just ignore 2017 data for now. There are other data issues as well.

In [131]:
rides2017.loc[rides2017['From station id'].isin(missing_stations), 'From station name'].drop_duplicates().tolist()

['Transit',
 'Missing',
 'Willmar-Schwabe-Str. / Jahnallee (LVB Mobilitätsstation 9) ',
 'Hauptbahnhof / Westhalle',
 'Straßenbhf. Angerbrücke (LVB Mobilitätsstation 14) ',
 'Jahnallee / Thomasiusstr. / Denkmal',
 'Am Hallischen Tor / Brühl',
 'Schillerstr. / Universitätsstr. / Mensa',
 'Scheffelstr. / Karl-Liebknecht-Str. (LVB Mobilitätsstation 16) ',
 'nextbike IT Peter Touch',
 'S-Bhf. Gohlis (LVB Mobilitätsstation 13) ',
 'Hauptbahnhof / Westseite (LVB Mobilitätsstation 4)',
 'Nordplatz (LVB Mobilitätsstation 8)',
 'Industriestr. / Karlbrücke',
 'Lene-Voigt-Park / Albert-Schweitzer-Str.',
 'Forbes and Gist (Open Streets May 2017)',
 'Westplatz ',
 'Leipzig International School (Könneritzstr. / Alfred-Frank-Str.)',
 'Markgrafenstr. / LVB Servicecenter (LVB Mobilitätsstation 1) ',
 'nextbike IT Daniel Desk',
 'Healthy Hauler',
 'ASCEND Pittsburgh',
 'nextbike IT Elsterstr.',
 'Western Ave & Bidwell Street',
 'S Main St & Alexander St',
 'Highmark Stadium']

## Popular Stations

We'll define popularity as average daily turnover.

Turnover is the daily number of leaving bikes subtracted from the daily number of arriving bikes.

### Cleaning

A 5-minute round trip seems a little suspicious to me.

In [191]:
one_way = rides2016['From station id'] == rides2016['To station id']
five_minute = rides2016['Tripduration'] < 300

failed = one_way & short
failed.sum()

1985

In [192]:
rides2016 = rides2016.loc[~failed]

In [193]:
leaving  = rides2016.groupby(['From station id', pd.Grouper(key='Starttime', freq='D')])['Trip id'].count()
arriving = rides2016.groupby(['To station id',   pd.Grouper(key='Stoptime' , freq='D')])['Trip id'].count()

leaving.index.names = arriving.index.names = ['station', 'date']

turnover = (arriving - leaving).mean(level='station')
turnover.name = 'turnover'
turnover.head()

station
1000    1.680782
1001    2.519355
1002   -0.248000
1003    0.068259
1004   -0.278169
Name: turnover, dtype: float64

In [194]:
stations2016 = stations.join(turnover)
stations2016

,Station Name,# of Racks,Latitude,Longitude,x,y,turnover
Station #,,,,,,,
1000,Liberty Ave & Stanwix St,16.0,40.441326,-80.004679,-8.906080e+06,4.930283e+06,1.680782
1001,Forbes Ave & Market Square,19.0,40.440877,-80.003080,-8.905902e+06,4.930218e+06,2.519355
1002,Third Ave & Wood St,15.0,40.439030,-80.001860,-8.905766e+06,4.929948e+06,-0.248000
1003,First Ave & Smithfield St (Art Institute),15.0,40.437200,-80.000375,-8.905601e+06,4.929680e+06,0.068259
...,...,...,...,...,...,...,...
1049,S 12th St & E Carson St,19.0,40.428661,-79.986358,-8.904041e+06,4.928431e+06,0.208075
1050,Healthy Ride Hub,2.0,40.461817,-79.968002,-8.901997e+06,4.933281e+06,-0.025641
1061,33rd St and Penn Ave,2.0,40.462026,-79.968114,-8.902010e+06,4.933311e+06,-0.500000
1060,Open Streets West End,NaN,40.440910,-80.034810,-8.909434e+06,4.930222e+06,-9.000000


## Mapping

<img src='https://www.fullstackpython.com/img/logos/bokeh.jpg' width='60%'>

*[Bokeh](https://bokeh.pydata.org/en/latest/) is a Python interactive visualization library that targets modern web browsers for presentation.*

First a little transformation I want to plot
* Every bike station
* The radius of each glyph is the absolute value of `'turnover'`
* The color indicates positive and negative

In [199]:
def scale(series):
    a = series.abs()
    return a / a.max() * 100

stations2016['size'] = scale(stations2016['turnover'])
stations2016['color'] = np.sign(stations2016['turnover']).map({-1:'red', 1:'green'})

In [196]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import HoverTool, WheelZoomTool
from bokeh.tile_providers import STAMEN_TONER
output_notebook()

Loading BokehJS ...

In [200]:
from pyproj import Proj, transform
inProj = Proj(init='EPSG:4326')
outProj = Proj(init='EPSG:3857')

stations2016['x'], stations2016['y'] = transform(inProj, outProj,
                                        stations2016['Longitude'].values,
                                        stations2016['Latitude'].values)

bounds = stations2016[['Longitude','Latitude']].median()

x, y = transform(inProj, outProj,
                             bounds['Longitude'],
                             bounds['Latitude'])
edge = 2200
x_range = [x-edge,x+edge]
y_range = [y-edge,y+edge]

Use your scoll, just like Google Maps to zoom.

In [201]:
plot = figure(x_range=x_range, y_range=y_range, plot_width=700, active_scroll='wheel_zoom')
plot.axis.visible = False

source = ColumnDataSource(stations2016)
hover = HoverTool(tooltips = [('Station','@{Station Name}'),('Turnover','@turnover')])

plot.add_tile(STAMEN_TONER, alpha=0.4)
plot.circle('x','y', color='color', size='size', alpha=0.8, source=source)
plot.add_tools(hover)
show(plot)

## Ride difficulty

We'll compute the following quantities
* ride distance
* elevation change
* deviation from expected duration

### elevation

In [ ]:
c_elev = googlemaps.Client(key=google_elev)

In [ ]:
pos = stations[['Latitude','Longitude']].values.tolist()
elev = [d['elevation'] for d in c_elev.elevation(pos)]
stations['elevation'] = elev

In [ ]:
def elevation_change(row, stations=stations):
    start = row['From station id']
    stop = row['To station id']
    
    e1 = stations.loc[start, 'elevation']
    e2 = stations.loc[stop, 'elevation']
    return e2 - e1

In [ ]:
rides['elevation_change'] = 0.0

one_way = rides['From station id'] != rides['To station id']
rides.loc[one_way,'elevation_change'] = rides.loc[one_way].apply(elevation_change, axis='columns')

In [ ]:
rides.groupby(np.sign(rides['elevation_change']))['Trip id'].count().plot.bar()

what are failed rides?

### distance

[Read this](https://gis.stackexchange.com/questions/84885/whats-the-difference-between-vincenty-and-great-circle-distance-calculations) for a discussion of distance algorithms.

<img src='http://numba.pydata.org/_static/numba_blue_icon_rgb.png' width='20%' align='right'>
<big><big><big><b>[Numba](http://numba.pydata.org)</b></big></big></big>

Numba will compile the function to optimized C code (using the LLVM compiler), and it understands NumPy arrays natively.

In [100]:
from geopy.distance import EARTH_RADIUS
from scipy.constants import mile
import numba

@numba.guvectorize(['(float64[:,:], float64[:,:], float64[:])'], '(n,m),(n,m)->(n)', nopython=True)
def _haversine(origin, destination, output):
    '''the haversine distance in miles'''
    assert origin.shape[0] == destination.shape[0]
    
    # earth radius in miles
    constant = EARTH_RADIUS * 1000/mile * 2
    p = np.pi / 180
    
    n = origin.shape[0]
    for i in range(n):
        lat1 = origin[i,0]
        lon1 = origin[i,1]
        lat2 = destination[i,0]
        lon2 = destination[i,1]
    
    
        a = 0.5 - np.cos((lat2 - lat1) * p) / 2  \
           + np.cos(lat1 * p) * np.cos(lat2 * p) \
           * (1 - np.cos((lon2 - lon1) * p )) / 2
        output[i] = constant * np.arcsin(np.sqrt(a))

def haversine(origin, destination, index='origin'):
    '''The haversine distance in miles
    
    Compute the distance in miles between arrays of origin
    and destination locations.
    
    Parameters
    ----------
    origin      : Pandas DataFrame of Latitude, Longitude positions
    destination : Pandas DataFrame of Latitude, Longitude positions
    index       : str
                  'origin' (default) - use origin's index on output
                  'destination'      - use destination's index on output
                   
    
    Returns
    -------
    Pandas Series on the same index as origin'''
    
    out = np.empty(shape=origin.shape[0], dtype=np.float64)
    haversine(origin.values, destination.values, out)
    
    index = origin.index if index == 'origin' else destination.index
    return pd.Series(out, index=index)

In [103]:
_haversine.signature

'(n,m),(n,m)->(n)'

In [108]:
%%time
start = rides2016.join(stations, on='From station id')[['Latitude','Longitude']]
end = rides2016.join(stations, on='To station id')[['Latitude', 'Longitude']]

distances = haversine(start, end)

#d_haversine = np.empty(shape=p0.shape[0], dtype=np.float64)

#haversine(p0, p1, d_haversine)

AttributeError: 'numpy.ndarray' object has no attribute 'values'

This function gives the same results as the slow `great_cirle()` from `geopy`.

In [ ]:
np.allclose(d_circle, d_haversine)

### grade

Ride difficulty will be defined as [percent grade](https://en.wikipedia.org/wiki/Grade_(slope)), which is dimensionless.

Bikes returned to the same station have `NaN` grade.

In [ ]:
rides['distance (miles)'] = d_vincenty

In [ ]:
rides['grade'] = rides['elevation_change'] / (rides['distance (miles)']*5280) * 100

In [ ]:
with pd.option_context('display.max_rows',15):
    print(rides['grade'].describe())

Fit a simple normal distribution.

In [ ]:
mu = rides['grade'].mean()
sigma = rides['grade'].std()
from scipy.stats import norm
rv = norm(loc=mu, scale=sigma)

Rides taken are centered near zero-grade trips. All rides, even those that are returned to the same station are included.

They almost follow a normal distribution.

In [ ]:
ax = rides['grade'].plot.hist(normed=True, cumulative=True, histtype='step', bins=100, figsize=(15,8))

x = np.linspace(-3, 3, 1000)
ax.plot(x, rv.cdf(x))

plt.xlabel('% Grade')

In [ ]:
by_ride = rides.loc[one_way].groupby(['From station id','To station id'])

popularity = by_ride['Trip id'].count()
popularity.name = 'rides'
difficulty = by_ride['grade'].first()

In [ ]:
plt.scatter(popularity, difficulty, alpha=0.5)
plt.xlabel('number of rides in 2016')
plt.ylabel('% Grade')

### outliers

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
X = pd.concat([popularity,difficulty], axis='columns')

In [ ]:
iso = IsolationForest(contamination=0.02)
iso.fit(X)
labels = iso.predict(X)
(labels==-1).sum()

In [ ]:
colors = np.where(labels==-1, 'red', 'blue')

plt.scatter(popularity, difficulty, alpha=0.5, c=colors)
plt.xlabel('number of rides in 2016')
plt.ylabel('% Grade')

In [ ]:
outliers = (
    X.loc[labels==-1]
    .reset_index()
    .join(stations['StationName'], on='From station id')
    .join(stations['StationName'], on='To station id', lsuffix='_from', rsuffix='_to')
)    

These are the outlier trips. They fall into two categories
1. Extremely popular rides
2. Rides with extreme grade

In [ ]:
pd.options.display.max_rows = 100
outliers.sort_values(['grade','rides'])

## Google Distance Matrix

Let's use the Google Maps Distance Matrix to get *actual* distance along the journey.

This assumes that the rider kept to Google Map's recommended route.

In [ ]:
c_dist = googlemaps.Client(key=google_dist)

In [ ]:
origin = (
    rides[['From station id']]
    .merge(stations[['Latitude','Longitude']], left_on='From station id', right_index=True)
)

destination = (
    rides[['To station id']]
    .merge(stations[['Latitude','Longitude']], left_on='To station id', right_index=True)
)

unique_pairs = origin.join(destination, lsuffix='_origin', rsuffix='_dest').drop_duplicates()

In [ ]:
unique_pairs.head()

In [ ]:
def g_dist_time(df):
    origins = df[['Latitude_origin','Longitude_origin']].to_frame().T.to_records(index=False)
    destinations = df[['Latitude_dest','Longitude_dest']].to_frame().T.to_records(index=False)
    
    json = c_dist.distance_matrix(origins, destinations, units='imperial', mode='bicycling')
    
    rows = [r['elements'][0] for r in json['rows']]
    # units don't matter; ['distance']['value'] is _always_ in meters
    values = {'distance (meters)':rows[0]['distance']['value'],
                  'time':pd.to_timedelta(rows[0]['duration']['value'], unit='s')}
    return pd.Series(values)

In order to comply with the [Distance Matrix API usage limits](https://developers.google.com/maps/documentation/distance-matrix/usage-limits) let's run the apply every 10 rows and wait a few seconds in between. 

In [ ]:
import time

results = []
for chunk in np.array_split(unique_pairs, 200):
    time.sleep(10)
    result = chunk.apply(g_dist_time, axis='columns')
    results.append(result)

In [ ]:
gmap = pd.concat(results)
gmap.to_csv('distances.csv')
gmap.head()

In [ ]:
gmap = pd.read_csv('distances.csv')
gmap.head()

In [ ]:
gmap_pairs = unique_pairs.join(gmap).rename(columns={'distance (meters)':'Gdistance (meters)'})

In [ ]:
rides_complete = rides.merge(gmap_pairs)
#rides_complete.to_csv('rides2016.csv', index=False)
rides_complete.head()

# Datetime

In [ ]:
for c in 'Starttime','Stoptime':
    rides_complete[c] = pd.to_datetime(rides_complete[c])

The tripduration is claimed to be in seconds, but many of these just look wrong.

In [ ]:
rides_complete['Tripduration'] = pd.to_timedelta(rides_complete['Tripduration'], unit='s')

The starttime should not be after the stoptime!

In [ ]:
wierd = rides_complete['Starttime'] > rides_complete['Stoptime']
rides_complete.loc[wierd]

According to Google the longest ride between any two stations is 45 minutes.

In [ ]:
rides_complete['time'] = pd.to_timedelta(rides_complete['time'])
rides_complete['time'].describe()

Let's ignore rides longer than 2 hours. It's likely that the bike sat stationary somewhere along the way.

In [ ]:
diff = (rides_complete['Stoptime'] - rides_complete['Starttime'])

extreme = diff > pd.to_timedelta(2, unit='h')
extreme.sum()

In [ ]:
one_way = rides_complete['From station id'] != rides_complete['To station id']

clean = ~wierd & ~extreme & one_way

In [ ]:
diff.loc[clean].describe()

In [ ]:
deviation = (diff.loc[clean] - rides_complete.loc[clean, 'time']).dt.total_seconds() / 60
rides_complete['time deviation (min)'] = deviation

In [ ]:
deviation.describe()

In [ ]:
times = pd.cut(deviation, bins=[-30, 0, 10, 60, 120], labels=['early', 'on_time', 'late', 'very_late'])

A majority of our rides are on time!

In [ ]:
by_time = rides_complete.loc[clean].groupby(times)
by_time['Trip id'].count().plot.bar()

There may not be much correlation between the deviation from the Google Map time and the grade.

In [ ]:
rides_complete.plot.scatter(x='grade', y='time deviation (min)', figsize=(12,8), alpha=0.4)

Here are some of the longest rides.

In [ ]:
large_deviation = (
   rides_complete.loc[clean]
  .groupby(['From station name','To station name'])['time deviation (min)']
  .median()
  .nlargest(15)
)                   

large_deviation = large_deviation.reset_index().merge(rides_complete[['From station name','To station name','grade']])
large_deviation.drop_duplicates(subset=['From station name','To station name'])